In [1]:
import langchain
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import textwrap
import simple_llm_cache
import llm_cache_stats_wrapper
import os

# In order to make it easy to run work projects and personal AI experiments, override these key values with the value of *_PERSONAL, if set.
if "OPENAI_API_KEY_PERSONAL" in os.environ:
    print("Using key from OPENAI_API_KEY_PERSONAL environment variable")
    os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY_PERSONAL"]
if "ANTHROPIC_API_KEY_PERSONAL" in os.environ:
    print("Using key from ANTHROPIC_API_KEY_PERSONAL environment variable")
    os.environ["ANTHROPIC_API_KEY"] = os.environ["ANTHROPIC_API_KEY_PERSONAL"]
if "GOOGLE_APPLICATION_CREDENTIALS_PERSONAL" in os.environ:
    print("Using key from GOOGLE_APPLICATION_CREDENTIALS_PERSONAL environment variable")
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.environ["GOOGLE_APPLICATION_CREDENTIALS_PERSONAL"]

verbose = False
temperature = 0.5

langchain.llm_cache = llm_cache_stats_wrapper.LlmCacheStatsWrapper(simple_llm_cache.SimpleLlmCache("llm-cache.json"))

def dump_cache_stats_since_last_call():
    print(langchain.llm_cache.get_cache_stats_summary())
    langchain.llm_cache.clear_cache_stats()

template = """Answer the following question as if you are a {character} character:
{question}
"""

prompt = PromptTemplate(
    input_variables=["character", "question"],
    template=template)

for model_name in [
    "gpt-3.5-turbo",
    "gpt-4",
    "gpt-4o-mini",
    "gpt-4o",
    #"claude-3-haiku-20240307",
    #"gemini-1.5-flash-preview-0514",
]:
    if model_name.startswith("gpt-"):
        llm = ChatOpenAI(
            temperature=temperature,
            model_name = model_name)
    elif model_name.startswith("claude-"):
        llm = ChatAnthropic(
            temperature=temperature,
            model_name = model_name)
    elif model_name.startswith("gemini-"):
        llm = VertexAI(
            temperature=temperature,
            model_name = model_name)

    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose)

    for trial in range(2):
        print(f"\n*** {model_name} trial {trial} ***")
        langchain.llm_cache.inner_cache.set_trial(trial)
        output = chain.predict(
            character="pirate",
            question="What is a neural network?")
        print(textwrap.fill(output, width=80))

print()
dump_cache_stats_since_last_call()

Using key from OPENAI_API_KEY_PERSONAL environment variable


C:\Users\forre\AppData\Local\Temp\ipykernel_25060\2689278037.py:61: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(



*** gpt-3.5-turbo trial 0 ***
Arrr, a neural network be like a ship's crew of brainy pirates workin' together
to solve problems and make decisions. They be usin' algorithms and data to learn
and improve their thinkin'. It be like havin' a compass in yer noggin, helpin'
ye navigate the treacherous seas of information. Aye, a neural network be a
powerful tool fer any savvy pirate lookin' to plunder the digital seas.

*** gpt-3.5-turbo trial 1 ***
Arrr matey, a neural network be like a crew of brainy buccaneers workin'
together to process information and learn from it. Just like me crew workin'
together to navigate the high seas and find the buried treasures, a neural
network be usin' algorithms to make decisions and solve problems. It be like
magic, but with more wires and less rum!

*** gpt-4 trial 0 ***
Arr matey, a neural network be like a crew of seafarers, each with their own
task, workin' together to navigate the vast seas of information. It's a system
of algorithms, akin to the h

In [2]:
# Use the async method to demonstrate that the cache works for sync and async calls
output = await chain.apredict(character="poet", question="What is a neural network?")
print(output)
print()
dump_cache_stats_since_last_call()

Ah, dear inquirer, gather 'round and lend your ear,
For I shall weave a tapestry of words, both bright and clear.
A neural network, thou dost seek to comprehend,
Is a marvel of our age, where thoughts and circuits blend.

Imagine, if you will, a web of silken threads,
Each one a tiny whisper in a grander tale that spreads.
These threads are neurons, nodes of light and fire,
Connected in a dance, a symphony of wire.

Like stars within the night, they sparkle and they gleam,
Each one a single note within a vast, unending dream.
They learn, they grow, they mimic human mind,
In patterns intricate and deep, solutions they do find.

From images and words, to numbers cold and stark,
They find the hidden meanings, they light the deepest dark.
They are the poets of the code, the dreamers of the machine,
In their silent, humming world, they see what’s yet unseen.

So, a neural network, dear friend, is not just gears and steel,
It’s a canvas of the future, where thoughts and circuits feel.
A marv